In [9]:
#generate mds projections of nodes layerwise, as determined by their per category rank scores
import os
import time
import torch
import pickle
import sys

os.chdir('/mnt/data/chris/dropbox/Research-Hamblin/Projects/cnn_subgraph_visualizer/prep_model_scripts')

from dissected_Conv2d import *
from data_loading_functions import *

sys.path.insert(0, os.path.abspath('../'))
sys.path.insert(0, os.path.abspath('../visualizer_scripts/'))

os.chdir('../')
from prep_model_parameters import output_folder
from visualizer_helper_functions import rank_file_2_df
os.chdir('./prep_model_scripts')

print('generating mds projection of nodes')

import numpy as np
import pandas as pd
from sklearn import manifold
from sklearn.metrics import euclidean_distances


categories_nodes_df = pd.read_csv('../prepped_models/%s/ranks/categories_nodes_ranks.csv'%output_folder)
overall_edge_df = rank_file_2_df('../prepped_models/%s/ranks/categories_edges/overall_edges_rank.pt'%output_folder)

misc_data = pickle.load(open('../prepped_models/%s/misc_graph_data.pkl'%output_folder,'rb'))
layer_nodes = misc_data['layer_nodes']
num_layers = misc_data['num_layers']
num_nodes = misc_data['num_nodes']
categories = misc_data['categories']
num_img_chan = misc_data['num_img_chan']
imgnode_positions = misc_data['imgnode_positions']
imgnode_colors = misc_data['imgnode_colors']
imgnode_names = misc_data['imgnode_names']


generating mds projection of nodes


In [12]:

#make wide version of nodes_df
def get_col(node_num, df = categories_nodes_df, idx = 'node_num', col = 'layer'):
    return df.loc[(df[idx] == node_num) & (df['category'] == df['category'].unique()[0]), col].item()

def add_norm_col(df,categories=categories,delete_columns = ['small_SPAN','overall']):
    for c in delete_columns:
        try:
            categories.remove(c)
        except:
            pass
    norms = []
    for index, row in df.iterrows():
        norm = 0
        for i,category in enumerate(categories):
            norm += row[category]**2
        norm = np.sqrt(norm)
        norms.append(norm)
    norms = np.array(norms)
    df['category_norm'] = norms
    return df

def gen_wide_df(rank_type,df=categories_nodes_df,delete_columns = ['small_SPAN','overall']):
    print('making wide version of df')
    nodes_wide_df = df.pivot(index = 'node_num',columns='category', values=rank_type)
    nodes_wide_df.reset_index(inplace=True)
    nodes_wide_df['layer'] = nodes_wide_df['node_num'].apply(get_col)
    nodes_wide_df = nodes_wide_df.rename(columns = {'category':'index'})
    for c in delete_columns:
        try:
            nodes_wide_df = nodes_wide_df.drop([c], axis=1)
        except:
            print('could drop column %s'%c)
    nodes_wide_df = add_norm_col(nodes_wide_df) 
    return nodes_wide_df

#rotation for mds plots
from scipy.spatial.distance import cdist

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)


def rotate_cartesian(vec2d,r):    #rotates 2d cartesian coordinates by some radians 
    x,y = vec2d[0], vec2d[1]
    x_out = np.sqrt(x**2+y**2)*np.cos(np.arctan2(y,x)+r)
    y_out = np.sqrt(x**2+y**2)*np.sin(np.arctan2(y,x)+r)
    return np.array([x_out,y_out])

def rotate_mds(layer_mds,rank_type,imgnode_positions=imgnode_positions,max_edges = 40,angles_tested=64):
    print('rotating layers to minimize edge lengths')
    for layer in range(len(layer_mds)):
        all_layer_positions = layer_mds[layer]
        layer_df = overall_edge_df.loc[(overall_edge_df['layer']==layer)].sort_values(rank_type+'_rank',ascending=False).head(max_edges)
        if layer == 0:
            all_prev_layer_positions = np.swapaxes(np.array([imgnode_positions['Y'],imgnode_positions['Z']]),0,1)
        else:
            all_prev_layer_positions = layer_mds[layer-1]
        #gen positions matrix for important edges
        select_layer_positions = []
        select_prev_layer_positions = []
        for row in layer_df.itertuples():
            select_layer_positions.append(all_layer_positions[row.out_channel])
            select_prev_layer_positions.append(all_prev_layer_positions[row.in_channel])
        #go through discrete rotations and find min distance
        min_dist = 10000000
        min_discrete_angle = 0
        for p in range(0,angles_tested):
            test_layer_positions=np.apply_along_axis(rotate_cartesian, 1, select_layer_positions,r=p*2*np.pi/angles_tested)
            dist = sum(np.diagonal(cdist(test_layer_positions,select_prev_layer_positions)))
            if dist < min_dist:
                min_discrete_angle = p
                min_dist = dist
        #update layer mds at layer by rotating by optimal angle
        print('rotating layer %s by %s rads'%(str(layer),str(min_discrete_angle*2*np.pi/angles_tested)))
        layer_mds[layer] = np.apply_along_axis(rotate_cartesian, 1, layer_mds[layer],r=min_discrete_angle*2*np.pi/angles_tested)
    return layer_mds 


def gen_layer_mds(nodes_df=categories_nodes_df):
    mds_projections ={}
    for rank_type in ['actxgrad','act','grad']:
        nodes_wide_df = gen_wide_df(rank_type+'_rank',df=nodes_df)
        layer_similarities = {}
        for layer in range(len(layer_nodes)):
            layer_df = nodes_wide_df[nodes_wide_df['layer'] == layer]
            for category in categories:
                layer_df[category] = layer_df.apply(lambda row : row[category]/row['category_norm'], axis = 1)   
            layer_similarities[layer] = euclidean_distances(layer_df.iloc[:,1:-2])

        layer_mds = {}
        for layer in layer_similarities:
            print('layer: %s'%str(layer))
            mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, 
              random_state=2, dissimilarity="precomputed", n_jobs=1)
            pos = mds.fit(layer_similarities[layer]).embedding_
            layer_mds[layer] = pos
        layer_mds = rotate_mds(layer_mds,rank_type)
        mds_projections[rank_type] = layer_mds
    return mds_projections


#grid layer projection

def gen_grid_positions():
    grid_projections = {}
    for layer in range(len(layer_nodes)):
        grid_projections[layer] = []
        num_nodes = len(layer_nodes[layer][1])
        if num_nodes == 1:
            grid_projections[layer] = np.array([[0,0]])
            continue
        elif num_nodes == 2:
            grid_projections[layer] = np.array([[.1,0],
                                                [-.1,0]])
            continue
        elif num_nodes == 3:
            grid_projections[layer] = np.array([[.1,.1],
                                                [0,0],
                                                [-.1,-.1]])
            continue
        elif num_nodes == 4:
            grid_projections[layer] = np.array([[.1,.1],
                                                [-.1,.1],
                                                [.1,-.1],
                                                [-.1,-.1]])
            continue
        elif num_nodes == 5:
            grid_projections[layer] = np.array([[.1,.1],
                                                [-.1,.1],
                                                [0,0],
                                                [.1,-.1],
                                                [-.1,-.1]])
            continue
        elif num_nodes == 6:
            grid_projections[layer] = np.array([[.1,.1],
                                                [0,.1],
                                                [-.1,.1],
                                                [.1,-.1],
                                                [0,-.1],
                                                [-.1,-.1]])
            continue
        elif num_nodes == 7:
            grid_projections[layer] = np.array([[.1,.1],
                                                [0,.1],
                                                [-.1,.1],
                                                [0,0],
                                                [.1,-.1],
                                                [0,-.1],
                                                [-.1,-.1]])
            continue
        elif num_nodes == 8:
            grid_projections[layer] = np.array([[.1,.1],
                                                [0,.1],
                                                [-.1,.1],
                                                [-.1,0],
                                                [.1,0],
                                                [.1,-.1],
                                                [0,-.1],
                                                [-.1,-.1]])  
            continue
        elif num_nodes == 9:
            grid_projections[layer] = np.array([[.1,.1],
                                                [0,.1],
                                                [-.1,.1],
                                                [-.1,0],
                                                [0,0],
                                                [.1,0],
                                                [.1,-.1],
                                                [0,-.1],
                                                [-.1,-.1]]) 
            continue
            
        elif num_nodes < 20:
            max_dis = .2
        elif num_nodes < 40:
            max_dis = .3
        elif num_nodes < 60:
            max_dis = .4
        elif num_nodes < 80:
            max_dis = .5
        elif num_nodes < 100:
            max_dis = .6
        elif num_nodes < 120:
            max_dis = .7
        elif num_nodes < 140:
            max_dis = .8
        else:
            max_dis = 1
        if np.floor(np.sqrt(num_nodes))*np.ceil(np.sqrt(num_nodes)) < num_nodes:
            x_spaces, y_spaces = np.ceil(np.sqrt(num_nodes)),np.ceil(np.sqrt(num_nodes))
        else:
            x_spaces, y_spaces = np.floor(np.sqrt(num_nodes)),np.ceil(np.sqrt(num_nodes))
        x = np.linspace(max_dis,-1*max_dis,int(x_spaces))
        y = np.linspace(max_dis,-1*max_dis,int(y_spaces))
        X,Y = np.meshgrid(x,y)
        X_flat = [item for sublist in X for item in sublist]
        Y_flat = [item for sublist in Y for item in sublist]
        for i in range(num_nodes):
            grid_projections[layer].append([X_flat[i],Y_flat[i]])    
        grid_projections[layer] = np.array(grid_projections[layer])
    return grid_projections


##Force Field equations
#center of mass
def points_2_com(points):
    #in_arr.shape = (num_points,dimensionality of points)
    #outputs center of mass as np.array([dim1,dim2, . . . ])
    outlist = []
    for dim in range(points.shape[1]):
        outlist.append(np.mean(points[:,dim]))
    return np.array(outlist)

def distance_and_direction(p1,p2):
    #returns a scalar magnitude and direction vector [x,y] from p1 [x,y] to p2[x,y]
    distance = np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
    direction = [p1[0]-p2[0],p1[1]-p2[1]]
    return distance, direction

def magnitude(point):
    return np.sqrt(np.sum(point**2))

def euc_2_polar(points):
    x,y = points[:,0], points[:,1]
    r = np.sqrt(x**2+y**2)
    t = np.arctan2(y,x)
    return r,t

def polar_2_euc(r,t):
    x = r*np.cos(t)
    y = r*np.sin(t)
    points = np.array([x,y]).transpose()
    return points

def scale_points(points,maximum = .25):
    r,t = euc_2_polar(points)
    max_r = np.max(r)
    scale = maximum/max_r
    r_scaled = scale*r
    scaled_points = polar_2_euc(r_scaled,t)
    return scaled_points

def translate_points(points,middle=0):
    x_avg = np.average(points[:,0])
    y_avg = np.average(points[:,1])
    points[:,0] = points[:,0] - x_avg + middle
    points[:,1] = points[:,1] - y_avg + middle
    return points

def scaling_forces(points):
    #in_arr.shape = (num_points,dimensionality of points)
    #outputs scalar forcing scaling factor
    distances = euclidean_distances(points)
    m = np.min(distances[np.triu_indices(points.shape[0], k = 1)])
    return m/2
    

def points_2_nonlin_repel(points, scaling):
    #in_arr.shape = (num_points,dimensionality of points)
    #outputs array of same shape with (num_points, dimensions of force vectors on those point)
    distances = euclidean_distances(points)
    mean_dis = np.mean(distances)
    #max_dis = np.max(distances)
    def force_on_point(point, points = points,scale = 1/mean_dis):
        dif = points - point
        dif_r, dif_t = euc_2_polar(dif)
        same_point_indices = np.where(dif_r == 0)[0]
        dif_r = np.delete(dif_r, same_point_indices, axis=0)
        dif_t = np.delete(dif_t, same_point_indices, axis=0)
        #magnitude = np.apply_along_axis(magnitude, 1, dif) 
        #force_mag = scaling*(1-scale*dif_r)
        #import pdb; pdb.set_trace()
        force_mag = 1/(dif_r)*mean_dis
        force_vecs = -1*polar_2_euc(force_mag,dif_t)
        #force_vecs = -1*np.delete(force_vecs, same_point_indices, axis=0)
        return np.sum(force_vecs,axis=0) 
    return np.apply_along_axis(force_on_point, 1, points)
    
    
def points_2_com_nonlin_attract(points,scaling):
    #in_arr.shape = (num_points,dimensionality of points)
    #outputs array of same shape with (num_points, dimensions of force vectors on those point)
    com = points_2_com(points)
    num_points = points.shape[0]
    def com_force_on_point(point, com = com, num_points = num_points,scaling=scaling):
        return [(com[0]-point[0])*scaling*num_points,(com[1]-point[1])*scaling*num_points]
    return np.apply_along_axis(com_force_on_point, 1, points)
    
    

def points_2_nonlin_force_field(points, scaling=1):
    #in_arr.shape = (num_points,dimensionality of points)
    #outputs array of same shape with (num_points, dimensions of force vectors on those point)
    scaling = scaling*scaling_forces(points)
    #return points_2_nonlin_repel(points,scaling)
    return points_2_nonlin_repel(points,scaling)+points_2_com_nonlin_attract(points,scaling)

def nonlinear_push_points(points,rate=.1):
    forces = points_2_nonlin_force_field(points)
    return points + rate*forces

def gen_layer_smooth_mds(mds_projections, iterations = 100,init_rate = .0001,step_ratio = 10,scale = .25):
    print('smoothing mds plots')
    init_rate = .0001
    smooth_mds_projection = {}
    #import pdb; pdb.set_trace()
    for rank_type in mds_projections.keys():
        smooth_mds_projection[rank_type] = {}
        for j in mds_projections[rank_type]:
            points = mds_projections[rank_type][j]
            for i in range(1,iterations):
                #move_points 
                #prev_points = deepcopy(points)
                points = nonlinear_push_points(points,rate=np.min([init_rate+i*init_rate/step_ratio,init_rate*10]))
            smooth_mds_projection[rank_type][j] = points
    
    max_points = 0
    for rank_type in mds_projections.keys():
        for i in mds_projections[rank_type]:
            points = mds_projections[rank_type][i]
            if points.shape[0] > max_points:
                max_points = points.shape[0]
                max_layer = i
        #get layer with maximum points and scale
        for i in mds_projections[rank_type]:
            if i == max_layer:
                smooth_mds_projection[rank_type][i] = scale_points(smooth_mds_projection[rank_type][i],maximum = scale)
            else:
                smooth_mds_projection[rank_type][i] = scale_points(smooth_mds_projection[rank_type][i],maximum = scale*np.sqrt(float(smooth_mds_projection[rank_type][i].shape[0])/float(max_points)))
        #rotate plots
        smooth_layer_mds = rotate_mds(smooth_mds_projection[rank_type],rank_type)
        smooth_mds_projection[rank_type] = smooth_layer_mds
    return smooth_mds_projection






def gen_layer_smooth_no_types(projections, iterations = 100,init_rate = .0001,step_ratio = 10,scale = .25,scale_same_size=True):
    print('smoothing plots')
    init_rate = .0001
    smooth_projection = {}
    #import pdb; pdb.set_trace()

    for j in range(len(projections)):
        points = projections[j]
        for i in range(1,iterations):
            #move_points 
            #prev_points = deepcopy(points)
            points = nonlinear_push_points(points,rate=np.min([init_rate+i*init_rate/step_ratio,init_rate*10]))
        smooth_projection[j] = points
    
    max_points = 0

    for i in range(len(projections)):
        points = projections[i]
        if points.shape[0] > max_points:
            max_points = points.shape[0]
            max_layer = i
    #get layer with maximum points and scale
    for i in range(len(projections)):
        if i == max_layer or scale_same_size:
            smooth_projection[i] = scale_points(smooth_projection[i],maximum = scale)
        else:
            smooth_projection[i] = scale_points(smooth_projection[i],maximum = scale*np.sqrt(float(smooth_projection[i].shape[0])/float(max_points)))
    #translate points to lie in center
    for i in range(len(projections)):
        smooth_projection[i] = translate_points(smooth_projection[i])
    #rotate plots
    smooth_rotated = rotate_no_types(smooth_projection)
    smooth_projection = smooth_rotated
    
    return smooth_rotated


def rotate_no_types(projections,rank_type='actxgrad',imgnode_positions=imgnode_positions,max_edges = 40,angles_tested=64):
    print('rotating layers to minimize edge lengths')
    for layer in range(len(projections)):
        all_layer_positions = projections[layer]
        layer_df = overall_edge_df.loc[(overall_edge_df['layer']==layer)].sort_values(rank_type+'_rank',ascending=False).head(max_edges)
        if layer == 0:
            all_prev_layer_positions = np.swapaxes(np.array([imgnode_positions['Y'],imgnode_positions['Z']]),0,1)
        else:
            all_prev_layer_positions = projections[layer-1]
        #gen positions matrix for important edges
        select_layer_positions = []
        select_prev_layer_positions = []
        for row in layer_df.itertuples():
            select_layer_positions.append(all_layer_positions[row.out_channel])
            select_prev_layer_positions.append(all_prev_layer_positions[row.in_channel])
        #go through discrete rotations and find min distance
        min_dist = 10000000
        min_discrete_angle = 0
        for p in range(0,angles_tested):
            test_layer_positions=np.apply_along_axis(rotate_cartesian, 1, select_layer_positions,r=p*2*np.pi/angles_tested)
            dist = sum(np.diagonal(cdist(test_layer_positions,select_prev_layer_positions)))
            if dist < min_dist:
                min_discrete_angle = p
                min_dist = dist
        #update layer mds at layer by rotating by optimal angle
        print('rotating layer %s by %s rads'%(str(layer),str(min_discrete_angle*2*np.pi/angles_tested)))
        projections[layer] = np.apply_along_axis(rotate_cartesian, 1, projections[layer],r=min_discrete_angle*2*np.pi/angles_tested)
    return projections

In [11]:
categories_nodes_df

,node_num,layer_name,layer,node_num_by_layer,act_rank,grad_rank,actxgrad_rank,category
0,0,features_0,0,0,0.325476,0.000014,0.000028,overall
1,1,features_0,0,1,0.609819,0.000016,0.000036,overall
2,2,features_0,0,2,0.440970,0.000035,0.000051,overall
3,3,features_0,0,3,0.357837,0.000037,0.000042,overall
4,4,features_0,0,4,0.518574,0.000029,0.000048,overall
...,...,...,...,...,...,...,...,...
1154299,1147,features_10,4,251,0.221436,0.000034,0.000043,zucchini
1154300,1148,features_10,4,252,0.246030,0.000048,0.000053,zucchini
1154301,1149,features_10,4,253,0.436829,0.000049,0.000081,zucchini
1154302,1150,features_10,4,254,0.142587,0.000018,0.000023,zucchini


In [5]:
nodes_wide_df = gen_wide_df('actxgrad_rank',df=categories_nodes_df)

making wide version of df


KeyboardInterrupt: 

In [ ]:
nodes_wide_df.columns

In [ ]:
layer = 4
layer_df = nodes_wide_df[nodes_wide_df['layer'] == layer]
layer_array = layer_df.iloc[:,1:-2].to_numpy()
scaled_layer_array = StandardScaler().fit_transform(layer_array)
scaled_layer_array.shape
  

In [ ]:
embedding = reducer.fit_transform(scaled_layer_array)
embedding.shape

In [37]:
import plotly.express as px


fig = px.scatter(
    embedding, x=0, y=1,
)
fig.show()

In [38]:
from copy import deepcopy

layer_df_normed = deepcopy(layer_df)
for category in categories:
    layer_df_normed[category] = layer_df_normed.apply(lambda row : row[category]/row['category_norm'], axis = 1) 
    
layer_array_normed = layer_df_normed.iloc[:,1:-2].to_numpy()
scaled_layer_array_normed = StandardScaler().fit_transform(layer_array_normed)
print(scaled_layer_array_normed.shape)
     
embedding_normed = reducer.fit_transform(scaled_layer_array_normed)
embedding_normed.shape


fig = px.scatter(
    embedding_normed, x=0, y=1,
)
fig.show()

(256, 1000)


In [4]:
penguins = pd.read_csv("https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv")
penguins.head()

,species_short,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [5]:
penguins = penguins.dropna()
penguins.species_short.value_counts()

Adelie       146
Gentoo       120
Chinstrap     68
Name: species_short, dtype: int64

In [6]:
import umap
reducer = umap.UMAP()

In [9]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [10]:
penguin_data = penguins[
    [
        "culmen_length_mm",
        "culmen_depth_mm",
        "flipper_length_mm",
        "body_mass_g",
    ]
].values
scaled_penguin_data = StandardScaler().fit_transform(penguin_data)

In [11]:
scaled_penguin_data.shape

(334, 4)

In [12]:
embedding = reducer.fit_transform(scaled_penguin_data)
embedding.shape

(334, 2)

In [ ]:
rank_type = 'actxgrad'
nodes_wide_df = gen_wide_df(rank_type+'_rank',df=nodes_df)

In [1]:


#fetch command line argument (prepped model)
#%reset
import sys
import os
from copy import deepcopy
import pickle
import torch
import plotly.offline as py
import plotly.graph_objs as go

os.chdir('/mnt/data/chris/dropbox/Research-Hamblin/Projects/cnn_subgraph_visualizer/')



sys.path.insert(0, os.path.abspath('./prep_model_scripts/'))
sys.path.insert(0, os.path.abspath('./visualizer_scripts/'))
from visualizer_helper_functions import *
from contrast_helper_functions import *
from featureviz_helper_functions import *
from subgraph_helper_functions import *
from ablation_functions import *
from receptive_field import *
from dissected_Conv2d import *
from copy import deepcopy


py.init_notebook_mode(connected=True)   #jupyter notebook only

prepped_model_folder = 'alexnet_sparse'    #set this to a subfolder of prunned_models

full_prepped_model_folder = os.path.abspath('prepped_models/%s'%prepped_model_folder)

possible_models = os.listdir('prepped_models')
print('possible models to visualizer are:')
print(possible_models)

print('\nYou\'ve chosen to visualize %s'%prepped_model_folder)


sys.path.insert(0,'prepped_models/%s'%prepped_model_folder)

import prep_model_params_used as prep_model_params

params = {}
params['prepped_model'] = prepped_model_folder
params['prepped_model_path'] = full_prepped_model_folder
params['device'] = 'cuda:0'
params['deepviz_neuron'] = True
params['deepviz_edge'] = False

#Parameters

#Non-GUI parameters

#deepviz
params['deepviz_param'] = None
params['deepviz_optim'] = None
params['deepviz_transforms'] = None
params['deepviz_image_size'] = prep_model_params.deepviz_image_size

#backend
params['cuda'] = prep_model_params.cuda    #use gpu acceleration when running model forward
params['input_image_directory'] = prep_model_params.input_img_path+'/'   #path to directory of imput images you want fed through the network
params['preprocess'] = prep_model_params.preprocess     #torchvision transfrom to pass input images through
params['label_file_path'] = prep_model_params.label_file_path
params['criterion'] = prep_model_params.criterion
params['rank_img_path'] = prep_model_params.rank_img_path
params['num_workers'] = prep_model_params.num_workers
params['seed'] = prep_model_params.seed
params['batch_size'] = prep_model_params.batch_size
#params['dynamic_act_cache_num'] = 4  #max number of input image activations 'dynamic_activations' will have simultaneously

 
#aesthetic 

params['node_size'] = 12
params['edge_size'] = 1
params['max_node_inputs'] = 10    #there is a dropdown showing the top weighted edge inputs to nodes, how many maps in dropdown?
params['layer_colors'] = ['rgba(31,119,180,', 
                          'rgba(255,127,14,',
                          'rgba(44,160,44,', 
                          'rgba(214,39,40,',
                          'rgba(39, 208, 214,', 
                          'rgba(242, 250, 17,',
                          'rgba(196, 94, 255,',
                          'rgba(193, 245, 5,',
                          'rgba(245, 85, 5,',
                          'rgba(5, 165, 245,',
                          'rgba(245, 5, 105,',
                          'rgba(218, 232, 23,',
                          'rgba(148, 23, 232,',
                          'rgba(23, 232, 166,',]


#visibility

params['show_ablations'] = False    #show network ablations modules
params['show_act_map_means'] = False #show mean value under activation maps
params['show_image_manip'] = False

#GUI parameters initialization (these parameters can be set in the GUI, but what values should they be initialized to?)
target_category = 'overall'     #category of images edges and nodes are weighted based on (which subgraph) 
rank_type = 'actxgrad'       #weighting criterion (actxgrad, act, grad, or weight)
projection = 'MDS smooth'           #how nodes within a layer are projected into the 2d plane (MDS or Grid)
edge_threshold = [.7,1]     #what range do edge ranks need to be in to be visualized
node_threshold = [.4,1]     #only relevant for hierarchical subgraph 

#### DONT EDIT BELOW initializations

figure_init = go.Figure()
figure_init.add_trace(go.Scatter(
            x=[],
            y=[]))
figure_init.update_layout(xaxis=dict(visible=False),
                  yaxis=dict(visible=False),
                  annotations = [dict(text="No Inputs",
                                      xref="paper",
                                      yref="paper",
                                      showarrow=False,
                                      font=dict(size=28))]
                 )

params['max_edge_weight'] = 1  #for the edge threshold slider, this dynamically adjusted its max value to max edge rank
                     #before there were multiple rank criterions, which made things confusing
                     #so well just fix it to 1 for now
        
        
display_dict = {True:'block',False:'none'}


#load Model

model_dis = dissect_model(deepcopy(prep_model_params.model),store_ranks=True,clear_ranks=True,cuda=params['cuda'],device=params['device']) #version of model with accessible preadd activations in Conv2d modules 
if params['cuda']:
    model_dis.cuda()
model_dis = model_dis.eval()    
model_dis.to(params['device'])

print('loaded model:')
print(prep_model_params.model)
        
#del prep_model_params.model
model = prep_model_params.model
if params['cuda']:
    model.cuda()
model = model.eval()
model.to(params['device'])


#load misc graph data
print('loading misc graph data')
misc_data = pickle.load(open('./prepped_models/%s/misc_graph_data.pkl'%prepped_model_folder,'rb'))
params['layer_nodes'] = misc_data['layer_nodes']
params['num_layers'] = misc_data['num_layers']
params['num_nodes'] = misc_data['num_nodes']
params['categories'] = misc_data['categories']
params['num_img_chan'] = misc_data['num_img_chan']
params['imgnode_positions'] = misc_data['imgnode_positions']
params['imgnode_colors'] = misc_data['imgnode_colors']
params['imgnode_names'] = misc_data['imgnode_names']
params['prepped_model_path'] = full_prepped_model_folder
params['ranks_data_path'] = full_prepped_model_folder+'/ranks/'






possible models to visualizer are:
['mnist_resize', 'alexnet_old.tar.gz', 'mnist_old', '.keep', 'mnist', 'old', 'alexnet10', 'letter_mixed_not_trained', 'imagenet10_sparse', 'alexnet_sparse_test', '.DS_Store', 'alexnet_sparse_full_model_10classes', 'alexnet_sparse.tar.gz', 'imagenet10', 'alexnet_sparse', 'alexnet_lucent', 'mnist.tgz', 'alexnet', 'letter_mixed', 'googlenet10_test', '._.DS_Store', 'alexnet_corrupted', 'alexnet.tar.gz']

You've chosen to visualize alexnet_sparse


/home/chris/miniconda3/envs/viscnn/lib/python3.7/site-packages/torch/cuda/__init__.py:125: UserWarning:


Tesla K40c with CUDA capability sm_35 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the Tesla K40c GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/




loaded model:
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096

In [15]:
d = {True:'neuron',False:'channel'}

d[True]

'neuron'

In [3]:
import pickle
# def get_model_activations_from_image(image_path, model_dis, params):
# 	print('running model to fetch activations')
# 	cuda = params['cuda']
# 	model_dis = set_across_model(model_dis,'target_node',None)
# 	if 'device' in params.keys():
# 		model_dis.to(params['device'])
# 	#image loading 
# 	image = preprocess_image(image_path,params)
# 	image_name = image_path.split('/')[-1]
# 	#pass image through model
# 	output = model_dis(image)
# 	#recursively fectch activations in conv2d_dissected modules
# 	layer_activations = get_activations_from_dissected_Conv2d_modules(model_dis)
# 	#return correctly formatted activation dict
# 	return act_array_2_imgname_dict(layer_activations,[image_name])
def object_2_str(obj,prefix):
    if isinstance(obj,str):
        return obj
    if isinstance(obj,list):
        return obj
    if obj is None:
        return('none')
    else:
        s= inspect.getsource(obj).replace(' ','').replace(prefix,'').strip()
        return inspect.getsource(obj)


    
##This function is horribly unoptimized, should not have to run through dissected model, and certainly should have to run 
##one image at a time
def gen_deepviz_neuron_activations(model_dis,params):
    cuda = params['cuda']
    model_dis = set_across_model(model_dis,'target_node',None)
    if 'device' in params.keys():
        model_dis.to(params['device'])
    activations = []
    
    file_path = params['prepped_model_path']+'/visualizations/images.csv'
    images_df = pd.read_csv(file_path,dtype=str)
    for layer_num in range(len(params['layer_nodes'])):
        #initialize activation arrays to zeros
        activations.append(np.zeros([len(params['layer_nodes'][layer_num][1]),len(params['layer_nodes'][layer_num][1])]))
        for within_layer_nodeid, nodeid in enumerate(params['layer_nodes'][layer_num][1]):
            objective_str = gen_objective_str(str(nodeid),model,params)
            parametrizer = params['deepviz_param']
            optimizer = params['deepviz_optim']
            transforms = params['deepviz_transforms']
            image_size = params['deepviz_image_size']
            param_str = object_2_str(parametrizer,"params['deepviz_param']=")
            optimizer_str = object_2_str(optimizer,"params['deepviz_optim']=")
            transforms_str = object_2_str(transforms,"params['deepviz_transforms']=")
            df_sel = images_df.loc[(images_df['targetid'] == str(nodeid)) & (images_df['objective'] == objective_str) & (images_df['parametrizer'] == param_str) & (images_df['optimizer'] == optimizer_str) & (images_df['transforms'] == transforms_str) & (images_df['neuron'] == 'True')]
            image_name = df_sel.iloc[0]['image_name']
            print('found image %s'%image_name)
            image_path = params['prepped_model_path']+'/visualizations/images/neuron/%s'%image_name
            image = preprocess_image(image_path,params)
            output = model_dis(image)
            layer_activations = get_activations_from_dissected_Conv2d_modules(model_dis)
            target_pixels = np.unravel_index(np.argmax(layer_activations['nodes'][layer_num][0][within_layer_nodeid], axis=None), layer_activations['nodes'][layer_num][0][within_layer_nodeid].shape)
            for other_within_layer_nodeid, other_nodeid in enumerate(params['layer_nodes'][layer_num][1]):
                activations[-1][int(other_within_layer_nodeid)][within_layer_nodeid] = layer_activations['nodes'][layer_num][0][other_within_layer_nodeid][target_pixels[0]][target_pixels[1]]
    pickle.dump(activations,open(params['prepped_model_path']+'/deepviz_activations_for_projection.pkl','wb'))            
    return activations
            
# def run_deepviz_through_model(model,edgeid,params):
#     model = set_across_model(model,'target_node',None)
#     objective_str = gen_objective_str(edgeid,model,params)
#     file_path = params['prepped_model_path']+'/visualizations/images.csv'
#     parametrizer = params['deepviz_param']
#     optimizer = params['deepviz_optim']
#     transforms = params['deepviz_transforms']
#     image_size = params['deepviz_image_size']
#     neuron = params['deepviz_neuron']

#     objective = gen_objective(edgeid,model,params,neuron)
#     param_str = object_2_str(parametrizer,"params['deepviz_param']=")
#     optimizer_str = object_2_str(optimizer,"params['deepviz_optim']=")
#     transforms_str = object_2_str(transforms,"params['deepviz_transforms']=")
#     df = pd.read_csv(file_path,dtype=str)

#     nodeid = edgeid.split('-')[0]
#     if 'r' in edgeid or 'g' in edgeid or 'b' in edgeid or 'gs' in edgeid:
#         df_sel = df.loc[(df['targetid'] == str(edgeid)) & (df['objective'] == objective_str) & (df['parametrizer'] == param_str) & (df['optimizer'] == optimizer_str) & (df['transforms'] == transforms_str) & (df['neuron'] == str(neuron))]
#         if len(df_sel) == 0:
#             print('deepviz image not found for node input %s, generating . . .'%edgeid)
#             #image_name = 'deepviz_'+str(targetid)+'_'+objective+'_'+str(time.time())+'.jpg'
#             image_name = str(edgeid)+'_'+objective_str+'_'+str(time.time())+'.jpg'
#             gen_visualization(model,image_name,objective,parametrizer,optimizer,transforms,image_size,neuron,params) 
#             with open(file_path, 'a') as csv:
#                 csv.write(','.join([image_name,str(edgeid),objective_str,param_str,optimizer_str,transforms_str,str(neuron)])+'\n') 
#         else:       
#             print('found pre-generated edge image')
#             image_name = df_sel.iloc[0]['image_name']

In [4]:
activations = pickle.load(open(params['prepped_model_path']+'/deepviz_activations_for_projection.pkl','rb'))

In [5]:
import umap

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


reducer = umap.UMAP()

embeddings = []

for i, layer in enumerate(activations):
    print(i)
    layer_scaled = StandardScaler().fit_transform(layer)
    embedding = reducer.fit_transform(layer_scaled)
    embeddings.append(embedding)
    


0
1
2
3
4


In [6]:
import plotly.express as px
fig = px.scatter(
    embeddings[2], x=0, y=1,
)
fig.show()

In [13]:
smooth_embeddings = gen_layer_smooth_no_types(embeddings,iterations=200,scale=.3)

smoothing mds plots
rotating layers to minimize edge lengths
rotating layer 0 by 6.086835766330224 rads
rotating layer 1 by 1.2762720155208536 rads
rotating layer 2 by 5.792311455056181 rads
rotating layer 3 by 1.8653206380689396 rads
rotating layer 4 by 4.71238898038469 rads


In [24]:
smooth_embeddings[0].shape

(64, 2)

In [29]:
smooth_embeddings.keys()

dict_keys([0, 1, 2, 3, 4])

In [36]:

import plotly.express as px
fig = px.scatter(
    smooth_embeddings[2], x=0, y=1,
    width=600,height=600
)
fig.show()

In [14]:
#plot with images
layer = 0


trace=go.Scatter(x=embeddings[layer][:,0],
                 y=embeddings[layer][:,1],
                 mode='markers',
                   marker=dict(symbol='circle',
                         size=5,
                         opacity=.99
                         ),
           )

fig = go.Figure(data=trace)
# Add images

for i in range(embeddings[layer].shape[0]):
    nodeid = str(params['layer_nodes'][layer][1][i])
    fig.add_layout_image(
        dict(
            source="http://chrishamblin.xyz/images/viscnn_images/%s.jpg"%nodeid,
            x=embeddings[layer][i,0],
            y=embeddings[layer][i,1],
            ))
# fig.add_layout_image(dict(
#         source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/naphthalene.png",
#         x=1.0,
#         y=1.0,
#         )
# )
fig.update_layout_images(dict(
        xref="x",
        yref="y",
        sizex=0.2,
        sizey=0.2,
        xanchor="center",
        yanchor="middle"
))



fig.show()

In [16]:
#plot with images
layer = 2


trace=go.Scatter(x=smooth_embeddings[layer][:,0],
                 y=smooth_embeddings[layer][:,1],
                 mode='markers',
                   marker=dict(symbol='circle',
                         size=5,
                         opacity=.99
                         ),
           )

fig = go.Figure(data=trace)
# Add images

for i in range(smooth_embeddings[layer].shape[0]):
    nodeid = str(params['layer_nodes'][layer][1][i])
    fig.add_layout_image(
        dict(
            source="http://chrishamblin.xyz/images/viscnn_images/%s.jpg"%nodeid,
            x=smooth_embeddings[layer][i,0],
            y=smooth_embeddings[layer][i,1],
            ))
# fig.add_layout_image(dict(
#         source="https://raw.githubusercontent.com/michaelbabyn/plot_data/master/naphthalene.png",
#         x=1.0,
#         y=1.0,
#         )
# )
fig.update_layout_images(dict(
        xref="x",
        yref="y",
        sizex=0.008,
        sizey=0.008,
        xanchor="center",
        yanchor="middle"
))

fig.update_layout(width=1000,height=1000)



fig.show()

In [36]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
from dash.exceptions import PreventUpdate
#import utils.dash_reusable_components as drc
import flask
import os

import json

from dash.dependencies import Input, Output, State

from plotly.subplots import make_subplots

from flask_caching import Cache



#external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(external_stylesheets = external_stylesheets)


styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


theme =  {
    'dark': True,
    'detail': '#007439',
    'primary': '#00EA64',
    'secondary': '#6E6E6E',
}


app.layout = html.Div([
        html.Div(
            children = [
                html.Label('Image Size'),
                dcc.Slider(
                        id='img_size',
                        min=.001,
                        max=.05,
                        step=0.0005,
                        value = .008
                    )
                ], className="three columns",
                ),
                
            html.Div([
                dcc.Graph(
                    id='fig',
                    figure=fig
                )
                ], className= 'nine columns'
                ),
            ], className="row")




#Network Graph Figure
@app.callback(
    Output('fig', 'figure'),
    [Input('img_size', 'value')],
    [State('fig','figure')])
def update_figure(img_size, fig):
    for i in fig['layout']['images']:
        i['sizex'] = img_size
        i['sizey'] = img_size
    return fig

In [ ]:
app.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1617985068.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1617985068.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1617985573.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-component-suites/dash_daq/dash_daq.v0_5_0m1596869724.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2021 21:24:17] "GET /_dash-componen

In [38]:
layer = 2
params['layer_nodes'][layer][1][2]

258

In [30]:
all_positions = pickle.load(open('/mnt/data/chris/dropbox/Research-Hamblin/Projects/cnn_subgraph_visualizer/prepped_models/alexnet_sparse/node_positions.pkl','rb'))

In [37]:
#### FORMAT NODE POSITIONS

layer_distance = 1   # distance in X direction each layer is separated by
layer_offset = 0
node_positions = []
for layer in smooth_embeddings:
        node_positions.append({})
        node_positions[-1]['X'] = [] 
        node_positions[-1]['Y'] = [] 
        node_positions[-1]['Z'] = []  
        for i in range(len(smooth_embeddings[layer])): 
            node_positions[-1]['Y'].append(smooth_embeddings[layer][i][0])
            node_positions[-1]['Z'].append(smooth_embeddings[layer][i][1])
            node_positions[-1]['X'].append(layer_offset)
        layer_offset+=1*layer_distance

all_positions['Deepviz UMAP smooth'] = node_positions

In [38]:
pickle.dump(all_positions,open('/mnt/data/chris/dropbox/Research-Hamblin/Projects/cnn_subgraph_visualizer/prepped_models/alexnet_sparse/node_positions.pkl','wb'))

In [41]:
for layer in smooth_embeddings:
    print(layer)

0
1
2
3
4


In [53]:
import time
start = time.time()
deepviz_activations = gen_deepviz_neuron_activations(model_dis,params)
print(time.time()-start)

generating feature_viz objective string for 0
found image 0.jpg
generating feature_viz objective string for 1
found image 1.jpg
generating feature_viz objective string for 2
found image 2.jpg
generating feature_viz objective string for 3
found image 3.jpg
generating feature_viz objective string for 4
found image 4.jpg
generating feature_viz objective string for 5
found image 5.jpg
generating feature_viz objective string for 6
found image 6.jpg
generating feature_viz objective string for 7
found image 7.jpg
generating feature_viz objective string for 8
found image 8.jpg
generating feature_viz objective string for 9
found image 9.jpg
generating feature_viz objective string for 10
found image 10.jpg
generating feature_viz objective string for 11
found image 11.jpg
generating feature_viz objective string for 12
found image 12.jpg
generating feature_viz objective string for 13
found image 13.jpg
generating feature_viz objective string for 14
found image 14.jpg
generating feature_viz objecti

generating feature_viz objective string for 124
found image 124.jpg
generating feature_viz objective string for 125
found image 125.jpg
generating feature_viz objective string for 126
found image 126.jpg
generating feature_viz objective string for 127
found image 127.jpg
generating feature_viz objective string for 128
found image 128.jpg
generating feature_viz objective string for 129
found image 129.jpg
generating feature_viz objective string for 130
found image 130.jpg
generating feature_viz objective string for 131
found image 131.jpg
generating feature_viz objective string for 132
found image 132.jpg
generating feature_viz objective string for 133
found image 133.jpg
generating feature_viz objective string for 134
found image 134.jpg
generating feature_viz objective string for 135
found image 135.jpg
generating feature_viz objective string for 136
found image 136.jpg
generating feature_viz objective string for 137
found image 137.jpg
generating feature_viz objective string for 138


generating feature_viz objective string for 245
found image 245.jpg
generating feature_viz objective string for 246
found image 246.jpg
generating feature_viz objective string for 247
found image 247.jpg
generating feature_viz objective string for 248
found image 248.jpg
generating feature_viz objective string for 249
found image 249.jpg
generating feature_viz objective string for 250
found image 250.jpg
generating feature_viz objective string for 251
found image 251.jpg
generating feature_viz objective string for 252
found image 252.jpg
generating feature_viz objective string for 253
found image 253.jpg
generating feature_viz objective string for 254
found image 254.jpg
generating feature_viz objective string for 255
found image 255.jpg
generating feature_viz objective string for 256
found image 256.jpg
generating feature_viz objective string for 257
found image 257.jpg
generating feature_viz objective string for 258
found image 258.jpg
generating feature_viz objective string for 259


generating feature_viz objective string for 366
found image 366.jpg
generating feature_viz objective string for 367
found image 367.jpg
generating feature_viz objective string for 368
found image 368.jpg
generating feature_viz objective string for 369
found image 369.jpg
generating feature_viz objective string for 370
found image 370.jpg
generating feature_viz objective string for 371
found image 371.jpg
generating feature_viz objective string for 372
found image 372.jpg
generating feature_viz objective string for 373
found image 373.jpg
generating feature_viz objective string for 374
found image 374.jpg
generating feature_viz objective string for 375
found image 375.jpg
generating feature_viz objective string for 376
found image 376.jpg
generating feature_viz objective string for 377
found image 377.jpg
generating feature_viz objective string for 378
found image 378.jpg
generating feature_viz objective string for 379
found image 379.jpg
generating feature_viz objective string for 380


generating feature_viz objective string for 487
found image 487.jpg
generating feature_viz objective string for 488
found image 488.jpg
generating feature_viz objective string for 489
found image 489.jpg
generating feature_viz objective string for 490
found image 490.jpg
generating feature_viz objective string for 491
found image 491.jpg
generating feature_viz objective string for 492
found image 492.jpg
generating feature_viz objective string for 493
found image 493.jpg
generating feature_viz objective string for 494
found image 494.jpg
generating feature_viz objective string for 495
found image 495.jpg
generating feature_viz objective string for 496
found image 496.jpg
generating feature_viz objective string for 497
found image 497.jpg
generating feature_viz objective string for 498
found image 498.jpg
generating feature_viz objective string for 499
found image 499.jpg
generating feature_viz objective string for 500
found image 500.jpg
generating feature_viz objective string for 501


generating feature_viz objective string for 608
found image 608.jpg
generating feature_viz objective string for 609
found image 609.jpg
generating feature_viz objective string for 610
found image 610.jpg
generating feature_viz objective string for 611
found image 611.jpg
generating feature_viz objective string for 612
found image 612.jpg
generating feature_viz objective string for 613
found image 613.jpg
generating feature_viz objective string for 614
found image 614.jpg
generating feature_viz objective string for 615
found image 615.jpg
generating feature_viz objective string for 616
found image 616.jpg
generating feature_viz objective string for 617
found image 617.jpg
generating feature_viz objective string for 618
found image 618.jpg
generating feature_viz objective string for 619
found image 619.jpg
generating feature_viz objective string for 620
found image 620.jpg
generating feature_viz objective string for 621
found image 621.jpg
generating feature_viz objective string for 622


generating feature_viz objective string for 729
found image 729.jpg
generating feature_viz objective string for 730
found image 730.jpg
generating feature_viz objective string for 731
found image 731.jpg
generating feature_viz objective string for 732
found image 732.jpg
generating feature_viz objective string for 733
found image 733.jpg
generating feature_viz objective string for 734
found image 734.jpg
generating feature_viz objective string for 735
found image 735.jpg
generating feature_viz objective string for 736
found image 736.jpg
generating feature_viz objective string for 737
found image 737.jpg
generating feature_viz objective string for 738
found image 738.jpg
generating feature_viz objective string for 739
found image 739.jpg
generating feature_viz objective string for 740
found image 740.jpg
generating feature_viz objective string for 741
found image 741.jpg
generating feature_viz objective string for 742
found image 742.jpg
generating feature_viz objective string for 743


generating feature_viz objective string for 850
found image 850.jpg
generating feature_viz objective string for 851
found image 851.jpg
generating feature_viz objective string for 852
found image 852.jpg
generating feature_viz objective string for 853
found image 853.jpg
generating feature_viz objective string for 854
found image 854.jpg
generating feature_viz objective string for 855
found image 855.jpg
generating feature_viz objective string for 856
found image 856.jpg
generating feature_viz objective string for 857
found image 857.jpg
generating feature_viz objective string for 858
found image 858.jpg
generating feature_viz objective string for 859
found image 859.jpg
generating feature_viz objective string for 860
found image 860.jpg
generating feature_viz objective string for 861
found image 861.jpg
generating feature_viz objective string for 862
found image 862.jpg
generating feature_viz objective string for 863
found image 863.jpg
generating feature_viz objective string for 864


generating feature_viz objective string for 971
found image 971.jpg
generating feature_viz objective string for 972
found image 972.jpg
generating feature_viz objective string for 973
found image 973.jpg
generating feature_viz objective string for 974
found image 974.jpg
generating feature_viz objective string for 975
found image 975.jpg
generating feature_viz objective string for 976
found image 976.jpg
generating feature_viz objective string for 977
found image 977.jpg
generating feature_viz objective string for 978
found image 978.jpg
generating feature_viz objective string for 979
found image 979.jpg
generating feature_viz objective string for 980
found image 980.jpg
generating feature_viz objective string for 981
found image 981.jpg
generating feature_viz objective string for 982
found image 982.jpg
generating feature_viz objective string for 983
found image 983.jpg
generating feature_viz objective string for 984
found image 984.jpg
generating feature_viz objective string for 985


generating feature_viz objective string for 1089
found image 1089.jpg
generating feature_viz objective string for 1090
found image 1090.jpg
generating feature_viz objective string for 1091
found image 1091.jpg
generating feature_viz objective string for 1092
found image 1092.jpg
generating feature_viz objective string for 1093
found image 1093.jpg
generating feature_viz objective string for 1094
found image 1094.jpg
generating feature_viz objective string for 1095
found image 1095.jpg
generating feature_viz objective string for 1096
found image 1096.jpg
generating feature_viz objective string for 1097
found image 1097.jpg
generating feature_viz objective string for 1098
found image 1098.jpg
generating feature_viz objective string for 1099
found image 1099.jpg
generating feature_viz objective string for 1100
found image 1100.jpg
generating feature_viz objective string for 1101
found image 1101.jpg
generating feature_viz objective string for 1102
found image 1102.jpg
generating feature_v

In [51]:
deepviz_activations[0][3][3]

8.360067367553711

In [23]:
layer_activations['nodes'][0].shape

(1, 64, 55, 55)

In [33]:
layer_activations['nodes'][1].shape

(1, 192, 27, 27)

In [30]:
np.unravel_index(np.argmax(layer_activations['nodes'][0][0][0], axis=None), layer_activations['nodes'][0][0][0].shape)

(26, 25)

In [11]:
params['imgnode_positions']

{'X': [-1, -1, -1], 'Y': [0.0, 0.17, -0.17], 'Z': [0.2, -0.1, -0.1]}

In [23]:
v = np.array([[1,2],[3,4],[5,6]])
x_avg = np.average(v[:,0])
y_avg = np.average(v[:,1])
v[:,0] = v[:,0]-x_avg
v[:,1] = v[:,1]-y_avg
print(x_avg)
print(y_avg)
v

3.0
4.0


array([[-2, -2],
       [ 0,  0],
       [ 2,  2]])